# 爬下来的数据的存储和读取

## 回顾
- 爬取帖子列表页
- 爬取帖子详情页
- [新]存储帖子列表页的信息（标题、URL、作者、时间、阅读量）
- [新]对每一条帖子存储内容、所有评论

## 新知识
- 使用 pandas 操作 CSV
- 使用正则表达式进行内容匹配

In [28]:
!pip install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 24.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 20.6 MB/s eta 0:00:00


In [35]:
# import 基础包
import requests
from bs4 import BeautifulSoup

# 公共 header，可以参见 postman 生成的 Python 代码，这个通常每次使用的时候都是固定的
headers = {
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7,fr;q=0.6',
  'Cache-Control': 'max-age=0',
  'Connection': 'keep-alive',
  'Cookie': 'll="108288"; bid=EUIXgN0ZKpc; _pk_id.100001.8cb4=1f708fde13a7265b.1700533653.; __utmc=30149280; viewed="36165055"; douban-fav-remind=1; __yadk_uid=8ArcxBZuzhGsH9PICN0uVjxPJzsUPXf0; _ga=GA1.2.1030669451.1701136885; _gid=GA1.2.1551607325.1701136885; _ga_Y4GN1R87RG=GS1.1.1701136884.1.0.1701136900.0.0.0; __utmz=30149280.1701150588.3.2.utmcsr=localhost:8888|utmccn=(referral)|utmcmd=referral|utmcct=/; ap_v=0,6.0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1701180456%2C%22http%3A%2F%2Flocalhost%3A8888%2F%22%5D; _pk_ses.100001.8cb4=1; __utma=30149280.1324323786.1700533655.1701175760.1701180456.5; __utmt=1; __utmb=30149280.12.9.1701181143253; bid=qYTbTuF2dfs',
  'Referer': 'https://www.douban.com/group/596337/discussion?start=0&type=new',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-User': '?1',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
  'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"'
}
payload={}

# 列表页面函数

In [109]:
# 初始化一个新的 DataFrame 并指定列名
import pandas as pd
def query_page(group, start: int):
    url = "https://www.douban.com/group/{}/discussion?start={}&type=new".format(group,start)
    response = requests.request("GET", url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    arr = soup.select("td.title > a")
    times = soup.select("td.time")
    arr = soup.select("table.olt > tr:not(.th)")
    discussions = pd.DataFrame(columns=['url', 'title', 'author', 'read_count', 'time'])
    for item in arr:
        tds = item.find_all('td')
        href = tds[0].find("a")['href']
        title = tds[0].find("a")['title']
        author = tds[1].text.replace("\n", "")
        read_count = tds[2].text
        tm = tds[3].text
        # 向 Dataframe 追加行
        row = pd.DataFrame({
                'url': href,
                'title': title,
                'author': author, 
                'read_count': read_count,
                'time': tm,
            }, index=[0])
        discussions = pd.concat([discussions, row], ignore_index=True)
    return discussions
pd.concat([query_page(596337, 0),query_page(596337, 30)] , ignore_index=True)

,url,title,author,read_count,time
0,https://www.douban.com/group/topic/298479157/,求推荐好喝的豆浆粉,已注销,15,11-30 10:58
1,https://www.douban.com/group/topic/297924364/,有谁知道它叫啥,H,41,11-30 10:56
2,https://www.douban.com/group/topic/297936551/,于我而言，烘焙是安安静静的一个人的世界,吾非吾,22,11-30 10:55
3,https://www.douban.com/group/topic/298144110/,各位大厨们，麻烦帮我看看我买的是不是真牛肉，怎么这么像猪肉,momo,65,11-30 10:47
4,https://www.douban.com/group/topic/298483660/,初中生早餐🥣,金先生的花,6,11-30 10:40
5,https://www.douban.com/group/topic/298370485/,青椒，为什么我活了半辈子才明白有这么好吃的配菜,momo,71,11-30 10:37
6,https://www.douban.com/group/topic/298463910/,钓鱼归来,牧云放野,15,11-30 10:32
7,https://www.douban.com/group/topic/298409768/,炖鸡汤，怎样才能汤鲜肉不柴啊？,巴拿马是只猫,48,11-30 10:30
8,https://www.douban.com/group/topic/297841835/,不是我夸张，真香啊,Ocean,32,11-30 10:25
9,https://www.douban.com/group/topic/298463349/,看菜色，大概就知道哪里的饭菜了,豆友A9d4KPmYaI,5,11-30 10:10


# 帖子详情函数

In [120]:
import re
details = pd.DataFrame(columns=['url', 'type', 'text', 'imgs', 'author', 'time', 'location'])

def query_topic(details: pd.DataFrame, url: str,start=0):
    response =requests.request("GET", "{}?start={}".format(url, start), headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'html.parser')
    author_data = soup.select("span.from > a")
    author_create_time_data = soup.select("span.create-time.color-green")
    author_location_data = soup.select("span.ip-location")
    author_name, author_create_time, author_location = '', '', ''

    if len(author_data) > 0:
        author_name = author_data[0].text
    if len(author_create_time_data) > 0:
        author_create_time = author_create_time_data[0].text
    if len(author_location_data) > 0:
        author_location = author_location_data[0].text


    data = soup.select("div.rich-content.topic-richtext > p, div.rich-content.topic-richtext > div")
    # 下面的内容解析了文字也解析了图片，和视频里讲的不一样
    if len(data) > 0:
        for item in data:
            img_tags = item.select("img")
            img_urls = [img_tag["src"] for img_tag in img_tags]
            row = pd.DataFrame({
               'url': url,
               'type': '正文', 
               'text': item.text,
               'imgs': " ".join(img_urls),
               'author': author_name,
               'time': author_create_time,
               'location': author_location
            }, index=[0])
        details = pd.concat([details, row], ignore_index=True)
    arr = soup.select("div.reply-doc.content")
    for item in arr:
        img_tags = item.select("img")
        img_urls = [img_tag["src"] for img_tag in img_tags]
        txt = item.select("p.reply-content")[0].text
        author = item.select("h4 > a")[0].text
        pubtime = item.select("span.pubtime")[0].text
        pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})\s(.*)"
        match = re.match(pattern, pubtime)
        if match:
            time = match.group(1)
            location = match.group(2)
        row = pd.DataFrame({
               'url': url,
               'type': '评论', 
               'text': txt,
               'imgs': " ".join(img_urls),
               'author': author,
               'time': time,
               'location': location
            }, index=[0])
        details = pd.concat([details, row], ignore_index=True)
    if len(arr) >= 100:
        return query_topic(details, url, start+100)
    return details
query_topic(details, "https://www.douban.com/group/topic/288398770/", 0)  

,url,type,text,imgs,author,time,location
0,https://www.douban.com/group/topic/288398770/,正文,07.08,https://img2.doubanio.com/view/group_topic/l/p...,水微静,2023-05-13 12:11:48,江苏
1,https://www.douban.com/group/topic/288398770/,评论,07.10 毛豆烧小公鸡， 酸辣土豆丝（醋倒多了），青豆苗豆腐汤,https://img9.doubanio.com/view/richtext/large/...,水微静,2023-07-10 12:22:21,江苏
2,https://www.douban.com/group/topic/288398770/,评论,07.10 晚餐,https://img1.doubanio.com/view/richtext/large/...,水微静,2023-07-10 18:31:53,江苏
3,https://www.douban.com/group/topic/288398770/,评论,谢谢楼主的分享，可以不用想菜单了,,湮湮,2023-07-10 21:54:39,广东
4,https://www.douban.com/group/topic/288398770/,评论,😊,,水微静,2023-07-11 11:57:01,江苏
...,...,...,...,...,...,...,...
96,https://www.douban.com/group/topic/288398770/,评论,09.29 中秋，中午在家里吃。,https://img1.doubanio.com/view/richtext/large/...,水微静,2023-09-29 20:14:34,江苏
97,https://www.douban.com/group/topic/288398770/,评论,09.29 晚上到婆婆家，还是老公掌勺,https://img2.doubanio.com/view/richtext/large/...,水微静,2023-09-29 20:15:09,江苏
98,https://www.douban.com/group/topic/288398770/,评论,09.30 山姆腌制好的烤牛肉，直接烤，不方便烤用平底锅煎也一样，口味很好；烧了一碗辣辣的...,https://img2.doubanio.com/view/richtext/large/...,水微静,2023-09-30 18:45:55,江苏
99,https://www.douban.com/group/topic/288398770/,评论,10.01 国庆节，外甥女小夫妻来家里，吃得丰富一些，但还是家常。,https://img1.doubanio.com/view/richtext/large/...,水微静,2023-10-01 13:00:23,江苏


# Dataframe 与 csv 读写

In [121]:
discussions = pd.concat([query_page(596337, 0),query_page(596337, 30)] , ignore_index=True)
discussions.to_csv('discussions.csv', index=False)

discussions = pd.read_csv('discussions.csv')
for row in discussions.itertuples():
    details = pd.DataFrame(columns=['url', 'type', 'text', 'imgs', 'author', 'time', 'location'])
    details = query_topic(details, row.url, 0)
    pattern = r'/group/topic/(\d+)/'
    topic_id = row.url
    match = re.search(pattern, row.url)
    if match:
        topic_id = match.group(1)
    details.to_csv('topic_{}.csv'.format(topic_id), index=False)